In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from torch.autograd import Variable
import PIL
from matplotlib.pyplot import figure, imshow, axis
from matplotlib.image import imread
%matplotlib inline
from annoy import AnnoyIndex

import os
import cv2
import numpy as np
import tensorflow as tf
import sys
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from torch.autograd import Variable
import PIL
from os import listdir

# Import utilites
from utils import label_map_util
from utils import visualization_utils as vis_util

# Name of the directory containing the object detection module we're using
MODEL_NAME = 'inference_graph_fgo'
#file_dir = 'D:/object_detection/tf_model_dir/models/research/object_detection/images_fgo/test/'
file_dir = "D:/object_detection/tf_model_dir/models/research/object_detection/images_fgo_face/all_images/"

save_dir = 'D:/object_detection/tf_model_dir/models/research/object_detection/images_fgo_face/detected/'
crop_path = 'D:/object_detection/tf_model_dir/models/research/object_detection/images_fgo_face/heads/'
crop_holder = 'D:/object_detection/tf_model_dir/models/research/object_detection/images_fgo_face/crop_holder/'

#IMAGE_LOC = 'images_dice/train/'


imsize=224
### resnet stuff
extractor_resnet = models.resnet101(pretrained=True)
extractor_resnet.eval()
num_ftrs = 1000

loader = transforms.Compose([transforms.Resize((imsize,imsize)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

def image_loader(image_name):
    """load image, returns cuda tensor"""
    #image_loader_1 = PIL.Image.fromarray(image_name)
    image_loader_1 = PIL.Image.open(image_name).convert("RGB")
    image_loader_1 = loader(image_loader_1).float()
    image_loader_1 = Variable(image_loader_1, requires_grad=True)
    image_loader_1 = image_loader_1.unsqueeze(0)  #this is for VGG, may not be needed for ResNet
    y_pred = extractor_resnet(image_loader_1)
    return y_pred.tolist()[0]#.cuda()  #assumes that you're using GPU

def showImagesHorizontally(annoy_dict, base_img, dir_, list_of_files):
    fig = figure(figsize=(160, 160))
    number_of_files = len(list_of_files)
    list_of_files = [base_img] + list_of_files
    first_bool = 0 
    for i in range(number_of_files):
        if first_bool ==0:
            a=fig.add_subplot(1,number_of_files,i+1)
            image = imread(list_of_files[i])
            first_bool = 1
            
        else:
            a=fig.add_subplot(1,number_of_files,i+1)
            image = imread(dir_+annoy_dict[str(list_of_files[i])])
        
        #image = imread(dir_l+annoy_dict[str(list_of_files[i])])
        imshow(image,aspect='equal')
        axis('off')
    
    
annoy_index = AnnoyIndex(1000,metric='euclidean')
annoy_index.load('images_fgo_face/annoy_euclidean_fgo.ann')

dat = pd.read_csv('images_fgo_face/mapping_with_annoy.csv')

dict_fgo_annoy = {}
for index,row in dat.iterrows():
    dict_fgo_annoy[str(row['annoy_index'])] = str(row['original_image']) #switch this out

    

d:\users\micha\appdata\local\programs\python\python35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
D:\object_detection\tf_model_dir\models\research\object_detection\utils\visualization_utils.py:25: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "d:\users\micha\appdata\local\programs\python\python35\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "d:\users\micha\appdata\local\programs\python\python35\lib\runpy.py", line 85, in _run_code
  

In [ ]:

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")


# Grab path to current working directory
CWD_PATH = os.getcwd()

# Path to frozen detection graph .pb file, which contains the model that is used
# for object detection.
PATH_TO_CKPT = os.path.join(CWD_PATH,MODEL_NAME,'frozen_inference_graph.pb')

# Path to label map file
PATH_TO_LABELS = os.path.join(CWD_PATH,'training_fgo','labelmap.pbtxt')

# Path to image
#PATH_TO_IMAGE = os.path.join(CWD_PATH,IMAGE_NAME)

# Number of classes the object detector can identify
NUM_CLASSES = 1

# Load the label map.
# Label maps map indices to category names, so that when our convolution
# network predicts `5`, we know that this corresponds to `king`.
# Here we use internal utility functions, but anything that returns a
# dictionary mapping integers to appropriate string labels would be fine
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

# Load the Tensorflow model into memory.
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

    sess = tf.Session(graph=detection_graph)

# Define input and output tensors (i.e. data) for the object detection classifier

# Input tensor is the image
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

# Output tensors are the detection boxes, scores, and classes
# Each box represents a part of the image where a particular object was detected
detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

# Each score represents level of confidence for each of the objects.
# The score is shown on the result image, together with the class label.
detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')

# Number of objects detected
num_detections = detection_graph.get_tensor_by_name('num_detections:0')

target_dir = 'F:/fgo_multi_label/test/'
file_list = []
for i in listdir(target_dir):
    if 'xml' not in i[-4:]:
        file_list.append(i)
        

files_done = 0

for i in file_list:

    PATH_TO_IMAGE = target_dir+i
    
    image = cv2.imread(PATH_TO_IMAGE)
    #image = cv2.resize(image,(1920,1080))

    image_expanded = np.expand_dims(image, axis=0)
    try:
        image_for_cropping = image.copy()
    except:
        print('error with this image {}'.format(i))
        continue
    try:
        # Perform the actual detection by running the model with the image as input
        (boxes, scores, classes, num) = sess.run(
            [detection_boxes, detection_scores, detection_classes, num_detections],
            feed_dict={image_tensor: image_expanded})
    except:
        print('no idea?...')
        continue
    #print(boxes,scores,classes,num)
    # Draw the results of the detection (aka 'visulaize the results')

    image, class_list,box_list = vis_util.visualize_boxes_and_labels_on_image_array(
            image,
            np.squeeze(boxes),
            np.squeeze(classes).astype(np.int32),
            np.squeeze(scores),
            category_index,
            use_normalized_coordinates=True,
            line_thickness=8,
            min_score_thresh=0.80)

    for idx in range(len(box_list)):
            ymin = int(box_list[idx][0])
            xmin = int(box_list[idx][1])
            ymax = int(box_list[idx][2])
            xmax = int(box_list[idx][3])

            #class_number = cleaned_class_list[idx]
            #print(class_number,ymin,ymax,xmin,xmax)

            crop_img = image_for_cropping[ymin:ymax, xmin:xmax]
            cv2.imwrite(crop_holder+i[:-4]+'_'+str(idx)+i[-4:],crop_img)
            crop_img_name = crop_holder+i[:-4]+'_'+str(idx)+i[-4:]
            feature_vec = image_loader(crop_img_name)
            test_list = annoy_index.get_nns_by_vector(feature_vec, 5)

            dir_loc = 'D:/object_detection/tf_model_dir/models/research/object_detection/images_fgo_face/all_images/'

            showImagesHorizontally(dict_fgo_annoy,PATH_TO_IMAGE,dir_loc,test_list)      


d:\users\micha\appdata\local\programs\python\python35\lib\site-packages\matplotlib\pyplot.py:528: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [ ]:
#dat.to_csv('images_fgo_face/head_base_mapping.csv')